In [2]:
import nltk
import pandas as pd
import ast
import re
import numpy as np

import string 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

import unidecode
import nltk.corpus
from gensim.models import Word2Vec

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import config

[nltk_data] Error loading wordnet: <urlopen error [Errno 60] Operation
[nltk_data]     timed out>


In [9]:
df = pd.read_csv("Dataset/Cleaned_Indian_Food_Dataset.csv")

In [10]:
df.rename(columns = {'Cleaned-Ingredients':'ingredients'}, inplace = True)
for i in range(len(df["ingredients"])):
    df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
    df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
    df["ingredients"][i] = df["ingredients"][i].split(',')

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_30107/1364071601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i]=re.sub("\(.*?\)","",df["ingredients"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_30107/1364071601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ingredients"][i] = df["ingredients"][i].replace('(','').replace(')','')
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_30107/1364071601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [11]:
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       pass
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    remove_punctuations = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for each_item in ingredients:
        each_item.translate(remove_punctuations)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', each_item)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measures]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list

In [12]:
df['parsed'] = df.ingredients.apply(ingredient_parser)

In [13]:
df['parsed'] = df['parsed'].apply(lambda x: ', '.join(map(str, x)))

In [16]:
df.to_csv('string_ingredients.csv', index=False)